# Get Started

## Building a Panel App

```python
import panel as pn
pn.extension()
```

In [ ]:
import hvplot.pandas
import numpy as np
import pandas as pd
import panel as pn

In [ ]:
PRIMARY_COLOR = "#0072B5"
SECONDARY_COLOR = "#B54300"
CSV_FILE = (
    "https://raw.githubusercontent.com/holoviz/panel/main/examples/assets/occupancy.csv"

)

In [ ]:
pn.extension(design="material", sizing_mode="stretch_width")

In [ ]:
@pn.cache
def load_data():
    return pd.read_csv(CSV_FILE, parse_dates=["date"], index_col="date")

data   = load_data()
data.tail()

In [ ]:
def transform_data(column, window=30, sigma=10):
    avg = data[column].rolling(window=window).mean()
    residue = data[column] - avg
    std = residue.rolling(window=window).std()
    outliers = np.abs(residue) > sigma * std
    return avg, avg[outliers]

In [ ]:
def get_plot(column, window, sigma):
    avg, outliers = transform_data(column, window, sigma)
    return avg.hvplot(
        height=300, legend=False, color=PRIMARY_COLOR
    ) * outliers.hvplot.scatter(color=SECONDARY_COLOR, padding=0.1, legend=False)

In [ ]:
get_plot('Temperature', window=20, sigma=10)

In [ ]:
column_widget = pn.widgets.Select(name='Column', value='Temperature', options=list(data.columns))
window_widget = pn.widgets.IntSlider(name='Window', value=30, start=1, end=60)
sigma_widget = pn.widgets.IntSlider(name='Sigma', value=10, start=1, end=30)

In [ ]:
bound_plot = pn.bind(get_plot, column=column_widget, window=window_widget, sigma=sigma_widget)


In [ ]:
widgets = pn.Column(column_widget, window_widget, sigma_widget, sizing_mode='fixed', width=300)
pn.Column(widgets, bound_plot)

In [ ]:
pn.template.MaterialTemplate(
    site="Panel",
    title="Getting Started App",
    sidebar=[column_widget, window_widget, sigma_widget],
    main=[bound_plot],
).servable(); # The ; is needed in the notebook to not display the template. Its not needed in a script